# xgboost로 분류하기 <br>
1. 정답 못냈을 경우를 대비해 아무 숫자로나 적은 파일 만들어두고 시작하기 <br>
2. early stopping 하면 학습 오차 감소 안하면 stop 가능 <br>
n-estimator을 200으로 해둬도 초기 스탑을 50으로 설정하면 50회 반복하는 동안 학습오차 감소 없으면 스탑함

### 과적합 제어하는 법
1. etq(learning rate임, 0~1)를 낮춘다. eta 낮추면 num_boost_round는 높여야 함 <br>
num_boost_round: waek learner 개수
2. max_depth값 낮추기 
3. min_child_weight 높이기
4. gamma값 높이기
5. subsample, colsample_bytree 낮추기

### 순서
1. 데이터 분리하기  -> sklearn, model selection 이런거 있음
2. xgboost는 DMatrix로 형태 바꿔서 사용해야함,  split한 train, test 둘다 바꿔주기

  
https://injo.tistory.com/44

In [8]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

In [12]:
# 악성종양 0 , 양성은 1
df = load_breast_cancer()
X = df.data
Y = df.target
cancer_df = pd.DataFrame(data=X, columns = df.feature_names)
cancer_df['target'] =Y

In [30]:
from sklearn.model_selection import train_test_split
# 데이터 분리하기  -> sklearn, model selection 이런거 있음
#help(train_test_split)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2)
len(Xtrain)

455

In [42]:
import xgboost
help(xgboost.train)

Help on function train in module xgboost.training:

train(params: Dict[str, Any], dtrain: xgboost.core.DMatrix, num_boost_round: int = 10, *, evals: Union[Sequence[Tuple[xgboost.core.DMatrix, str]], NoneType] = None, obj: Union[Callable[[numpy.ndarray, xgboost.core.DMatrix], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = None, feval: Union[Callable[[numpy.ndarray, xgboost.core.DMatrix], Tuple[str, float]], NoneType] = None, maximize: Union[bool, NoneType] = None, early_stopping_rounds: Union[int, NoneType] = None, evals_result: Dict[str, Dict[str, Union[List[float], List[Tuple[float, float]]]]] = None, verbose_eval: Union[bool, int, NoneType] = True, xgb_model: Union[str, os.PathLike, xgboost.core.Booster, bytearray, NoneType] = None, callbacks: Union[Sequence[xgboost.callback.TrainingCallback], NoneType] = None, custom_metric: Union[Callable[[numpy.ndarray, xgboost.core.DMatrix], Tuple[str, float]], NoneType] = None) -> xgboost.core.Booster
    Train a booster with given parameters

In [48]:
#help(xgboost.DMatrix)
dtrain = xgboost.DMatrix(data = Xtrain, label = ytrain)
dtest = xgboost.DMatrix(data = Xtest, label = ytest)
params =  {'max_depth' : 3,
         'eta' : 0.1, 
         'objective' : 'binary:logistic',
         'eval_metric' : 'logloss',
         'early_stoppings' : 100 }
model = xgboost.train(params = params, dtrain = dtrain, evals = [(dtrain, 'train'), (dtest, 'eval')],
                     num_boost_round = 300)

[15:31:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stoppings" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-logloss:0.61072	eval-logloss:0.61524
[1]	train-logloss:0.54286	eval-logloss:0.55123
[2]	train-logloss:0.48448	eval-logloss:0.49737
[3]	train-logloss:0.43392	eval-logloss:0.44985
[4]	train-logloss:0.39056	eval-logloss:0.40978
[5]	train-logloss:0.35423	eval-logloss:0.37654
[6]	train-logloss:0.32107	eval-logloss:0.34534
[7]	train-logloss:0.29157	eval-logloss:0.31888
[8]	train-logloss:0.26728	eval-logloss:0.29667
[9]	train-logloss:0.24413	eval-logloss:0.27636
[10]	train-logloss:0.22396	eval-logloss:0.25670
[11]	train-logloss:0.20522	eval-logloss:0.24259
[12]	train

[161]	train-logloss:0.00715	eval-logloss:0.14957
[162]	train-logloss:0.00713	eval-logloss:0.14989
[163]	train-logloss:0.00711	eval-logloss:0.15005
[164]	train-logloss:0.00709	eval-logloss:0.14994
[165]	train-logloss:0.00707	eval-logloss:0.14997
[166]	train-logloss:0.00705	eval-logloss:0.14989
[167]	train-logloss:0.00703	eval-logloss:0.15026
[168]	train-logloss:0.00701	eval-logloss:0.15011
[169]	train-logloss:0.00700	eval-logloss:0.14990
[170]	train-logloss:0.00698	eval-logloss:0.15002
[171]	train-logloss:0.00696	eval-logloss:0.15037
[172]	train-logloss:0.00694	eval-logloss:0.15023
[173]	train-logloss:0.00692	eval-logloss:0.15038
[174]	train-logloss:0.00690	eval-logloss:0.15068
[175]	train-logloss:0.00688	eval-logloss:0.15060
[176]	train-logloss:0.00687	eval-logloss:0.15086
[177]	train-logloss:0.00685	eval-logloss:0.15097
[178]	train-logloss:0.00683	eval-logloss:0.15088
[179]	train-logloss:0.00681	eval-logloss:0.15111
[180]	train-logloss:0.00680	eval-logloss:0.15103
[181]	train-logloss:

In [50]:
preds = model.predict(dtest)

In [54]:
result = [1 if i >0.5 else 0 for i in preds]
result

[1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [58]:
sum(abs(ytest-result))/len(ytest)

0.043859649122807015

#  빅분기 작업형 2유형 기출 1

In [1]:
import pandas as pd
path = 'C:/Users/100do/python/승혜/archive/'
ytrain = pd.read_csv(path + 'y_train.csv')
xtrain = pd.read_csv(path + 'X_train_2.csv' )
xtest = pd.read_csv(path + 'X_test_2.csv')
xtest

,cust_id,total_buy,max_buy,payback,main_product,where,days,buy_per_day,weekend_come,buy_period
0,3500,70900400,22000000,4050000.0,골프,부산본점,13,1.461538,0.789474,26
1,3501,310533100,38558000,48034700.0,농산물,잠실점,90,2.433333,0.369863,3
2,3502,305264140,14825000,30521000.0,가공식품,본 점,101,14.623762,0.083277,3
3,3503,7594080,5225000,NaN,주방용품,부산본점,5,2.000000,0.000000,47
4,3504,1795790,1411200,NaN,수산품,청량리점,3,2.666667,0.125000,8
...,...,...,...,...,...,...,...,...,...,...
2477,5977,82581500,23976000,NaN,골프,부산본점,8,1.750000,0.642857,40
2478,5978,480000,480000,NaN,섬유잡화,광주점,1,1.000000,0.000000,0
2479,5979,260003790,25750000,NaN,남성 캐주얼,본 점,19,3.736842,0.915493,18
2480,5980,88991520,18120000,NaN,육류,본 점,5,3.600000,0.444444,60


1. 숫자형, text, 변수 분리 -> get_dummies to text

In [5]:
xtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cust_id       3500 non-null   int64  
 1   total_buy     3500 non-null   int64  
 2   max_buy       3500 non-null   int64  
 3   payback       1205 non-null   float64
 4   main_product  3500 non-null   object 
 5   where         3500 non-null   object 
 6   days          3500 non-null   int64  
 7   buy_per_day   3500 non-null   float64
 8   weekend_come  3500 non-null   float64
 9   buy_period    3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [10]:
x_str = xtrain[['main_product', 'where']]
x_num = xtrain[['total_buy','max_buy','payback','days','buy_per_day','weekend_come','buy_period']]
xtest_str = xtest[['main_product', 'where']]
xtest_num = xtest[['total_buy','max_buy','payback','days','buy_per_day','weekend_come','buy_period']]

In [11]:
x_str = pd.get_dummies(x_str)
xtest_str = pd.get_dummies(xtest_str)
print(xtest_str.head())
print(len(xtest_str.columns))

   main_product_가공식품  main_product_가구  main_product_건강식품  main_product_골프  \
0                  0                0                  0                1   
1                  0                0                  0                0   
2                  1                0                  0                0   
3                  0                0                  0                0   
4                  0                0                  0                0   

   main_product_구두  main_product_기타  main_product_남성 캐주얼  main_product_남성 트랜디  \
0                0                0                    0                    0   
1                0                0                    0                    0   
2                0                0                    0                    0   
3                0                0                    0                    0   
4                0                0                    0                    0   

   main_product_남성정장  main_product_농산물  ...  where

2. xtest_str, x_str get dummies 한 컬럼 맞추기 위해 concat 진행
 - align 사용
 - align 사용할 때 inner 하면 두데이터 동시에 같는 인덱스만 출력한다.
 - test, train 클래스 개수 맞춰줄 때만 원 핫 인코딩 사용 가능함

In [17]:
x_str, xtest_str = x_str.align(xtest_str, join = 'inner', axis = 1)
len(xtest_str.columns)

65

3. 수치형 데이터에서 결측치 제거. <br>
 - 결측치 제거 후 개수 많이 작아진거 아니면 mean으로 대체
 - 이상체를 상한, 하한 임계치 수치로 변환
 - drop 해도 인덱스가 그대로 있으니까 str이랑 맞추는거 가능할 거라 생각함


In [31]:
# payback이 na인거는 환불 안한거라는 뜻이니까 0으로 바꾸자.
x_num = x_num.fillna(0)
q3 = x_num.quantile(q = 0.75)
q1 = x_num.quantile(q = 0.25)
iqr = q3-q1
upper = q3 + iqr*1.5
lower = q1 - iqr*1.5 # 다 음수니까 아래는 안빼도 될듯
x_num_rm = x_num[ x_num < upper ]
x_num_rm = x_num_rm.fillna(upper)
x_num_rm

,total_buy,max_buy,payback,days,buy_per_day,weekend_come,buy_period
0,68282840.0,11264000.0,6605625.0,19.0,3.894737,0.527027,17.0
1,2136000.0,2136000.0,300000.0,2.0,1.500000,0.000000,1.0
2,3197000.0,1639000.0,0.0,2.0,2.000000,0.000000,1.0
3,16077620.0,4935000.0,0.0,18.0,2.444444,0.318182,16.0
4,29050000.0,24000000.0,0.0,2.0,1.500000,0.000000,64.0
...,...,...,...,...,...,...,...
3495,3175200.0,3042900.0,0.0,1.0,2.000000,1.000000,0.0
3496,29628600.0,7200000.0,6049600.0,8.0,1.625000,0.461538,40.0
3497,75000.0,75000.0,0.0,1.0,1.000000,0.000000,0.0
3498,1875000.0,1000000.0,0.0,2.0,1.000000,0.000000,39.0


In [34]:
practice this -> # https://hobby-weighted.tistory.com/157   

NameError: name 'x_num_str' is not defined